In [1]:
import pandas as pd
import numpy as np
import sqlite3
import os

In [2]:
# import csv file
df = pd.read_csv('Sample-Superstore_cleaned.csv', sep=',', encoding='utf-8', low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   row_id                9994 non-null   int64  
 1   order_id              9994 non-null   object 
 2   order_date            9994 non-null   object 
 3   year                  9994 non-null   int64  
 4   month                 9994 non-null   int64  
 5   day_of_week           9994 non-null   object 
 6   ship_date             9994 non-null   object 
 7   ship_mode             9994 non-null   object 
 8   customer_id           9994 non-null   object 
 9   customer_name         9994 non-null   object 
 10  segment               9994 non-null   object 
 11  customer_address_key  9994 non-null   object 
 12  country               9994 non-null   object 
 13  city                  9994 non-null   object 
 14  state                 9994 non-null   object 
 15  postal_code          

In [3]:
# Change ship_date to datetime format
df['ship_date'] = pd.to_datetime(df['ship_date'], format='%Y-%m-%d')
df['order_date'] = pd.to_datetime(df['order_date'], format='%Y-%m-%d')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   row_id                9994 non-null   int64         
 1   order_id              9994 non-null   object        
 2   order_date            9994 non-null   datetime64[ns]
 3   year                  9994 non-null   int64         
 4   month                 9994 non-null   int64         
 5   day_of_week           9994 non-null   object        
 6   ship_date             9994 non-null   datetime64[ns]
 7   ship_mode             9994 non-null   object        
 8   customer_id           9994 non-null   object        
 9   customer_name         9994 non-null   object        
 10  segment               9994 non-null   object        
 11  customer_address_key  9994 non-null   object        
 12  country               9994 non-null   object        
 13  city              

SETUP OF SQL DATABASE STRUCTURE

Table 1 - Customer

In [4]:
# start sqlite3 connection and creation of database
conn = sqlite3.connect("superstore2.db")

cur = conn.cursor()

# Create customer table
create_customer_table = """
   CREATE TABLE IF NOT EXISTS customer (
      customer_id TEXT NOT NULL PRIMARY KEY,
      customer_name TEXT NOT NULL,
      segment TEXT NOT NULL
   )
"""

cur.execute(create_customer_table)

cur.close()
conn.close()

In [5]:
# fill customer table with data from df
customer = df[["customer_id", "customer_name", "segment"]].copy()
customer.drop_duplicates(inplace=True)

# Check result
customer


,customer_id,customer_name,segment
0,CG-12520,Claire Gute,Consumer
2,DV-13045,Darrin Van Huff,Corporate
3,SO-20335,Sean O'Donnell,Consumer
5,BH-11710,Brosina Hoffman,Consumer
12,AA-10480,Andrew Allen,Consumer
...,...,...,...
8666,CJ-11875,Carl Jackson,Corporate
9209,RS-19870,Roy Skaria,Home Office
9399,SC-20845,Sung Chung,Consumer
9441,RE-19405,Ricardo Emerson,Consumer


In [6]:
# insert data into customer table
conn = sqlite3.connect("superstore2.db")

customer.to_sql(
   "customer", # the name of the table
   con=conn, # the connection variable
   index=False, # whether to include the index column
   if_exists="append" # define behavior if table already exists
)

# Export customer table to csv without index
customer = pd.read_sql_query("SELECT * FROM customer", conn)

# Ensure the 'Tableau' folder exists
os.makedirs("Tableau", exist_ok=True)

# Save the CSV file into the 'Tableau' folder
customer.to_csv("Tableau/customer.csv", index=False)

conn.close()



Table 2 - Geographical Data

In [7]:
# start sqlite3 connection and creation of database
conn = sqlite3.connect("superstore2.db")

cur = conn.cursor()

# Create customer table
create_geographical_data_table = """
   CREATE TABLE IF NOT EXISTS geographical_data (
      state TEXT NOT NULL PRIMARY KEY,
      country TEXT NOT NULL,
      region TEXT NOT NULL
      )
"""

cur.execute(create_geographical_data_table)

cur.close()
conn.close()

In [8]:
# fill customer_address table with data from df
geographical_data = df[["state", "country", "region"]].copy()
geographical_data.drop_duplicates(inplace=True)

# Check result
geographical_data

,state,country,region
0,Kentucky,United States,South
2,California,United States,West
3,Florida,United States,South
12,North Carolina,United States,South
13,Washington,United States,West
14,Texas,United States,Central
16,Wisconsin,United States,Central
17,Utah,United States,West
21,Nebraska,United States,Central
23,Pennsylvania,United States,East


In [9]:
# insert data into geographical_data table
conn = sqlite3.connect("superstore2.db")

geographical_data.to_sql(
   "geographical_data", # the name of the table
   con=conn, # the connection variable
   index=False, # whether to include the index column
   if_exists="append" # define behavior if table already exists
)


# Export geographical_data table to csv without index
geographical_data = pd.read_sql_query("SELECT * FROM geographical_data", conn)

# Save the CSV file into the 'Tableau' folder
geographical_data.to_csv("Tableau/geographical_data.csv", index=False)


conn.close()

Table 3 - Customer Address

In [10]:
# start sqlite3 connection and creation of database
conn = sqlite3.connect("superstore2.db")

cur = conn.cursor()

# Create customer_address table
create_customer_address_table = """
   CREATE TABLE IF NOT EXISTS customer_address (
      customer_address_key TEXT NOT NULL PRIMARY KEY,
      city TEXT NOT NULL,
      postal_code INT NOT NULL,
      state TEXT NOT NULL,
      FOREIGN KEY (state) REFERENCES geographical_data(state)
   )
"""

cur.execute(create_customer_address_table)

cur.close()
conn.close()

In [11]:
# fill customer_address table with data from df
customer_address = df[["customer_address_key", "city", "postal_code", "state"]].copy()
customer_address.drop_duplicates(inplace=True)

# Check result
customer_address

,customer_address_key,city,postal_code,state
0,CG-12520-42420-Henderson-Kentucky,Henderson,42420,Kentucky
2,DV-13045-90036-Los Angeles-California,Los Angeles,90036,California
3,SO-20335-33311-Fort Lauderdale-Florida,Fort Lauderdale,33311,Florida
5,BH-11710-90032-Los Angeles-California,Los Angeles,90032,California
12,AA-10480-28027-Concord-North Carolina,Concord,28027,North Carolina
...,...,...,...,...
9986,ML-17410-90008-Los Angeles-California,Los Angeles,90008,California
9987,RA-19885-30605-Athens-Georgia,Athens,30605,Georgia
9989,TB-21400-33180-Miami-Florida,Miami,33180,Florida
9990,DB-13060-92627-Costa Mesa-California,Costa Mesa,92627,California


In [12]:
# insert data into customer_address table
conn = sqlite3.connect("superstore2.db")

customer_address.to_sql(
   "customer_address", # the name of the table
   con=conn, # the connection variable
   index=False, # whether to include the index column
   if_exists="append" # define behavior if table already exists
)

# Export customer_address table to csv without index
customer_address = pd.read_sql_query("SELECT * FROM customer_address", conn)

# Save the CSV file into the 'Tableau' folder
customer_address.to_csv("Tableau/customer_address.csv", index=False)

conn.close()


Table 4 - Product Categories

In [13]:
# start sqlite3 connection and creation of database
conn = sqlite3.connect("superstore2.db")

cur = conn.cursor()

# Create product_categories table
create_product_categories_table = """
   CREATE TABLE IF NOT EXISTS product_categories (
      subcategory TEXT NOT NULL PRIMARY KEY,
      category TEXT NOT NULL
   )
"""

cur.execute(create_product_categories_table)

cur.close()
conn.close()

In [14]:
# fill product_categories table with data from df
product_categories = df[["subcategory", "category"]].copy()
product_categories.drop_duplicates(inplace=True)

# Check result
product_categories

,subcategory,category
0,Bookcases,Furniture
1,Chairs,Furniture
2,Labels,Office Supplies
3,Tables,Furniture
4,Storage,Office Supplies
5,Furnishings,Furniture
6,Art,Office Supplies
7,Phones,Technology
8,Binders,Office Supplies
9,Appliances,Office Supplies


In [15]:
# insert data into product_categories table
conn = sqlite3.connect("superstore2.db")

product_categories.to_sql(
   "product_categories", # the name of the table
   con=conn, # the connection variable
   index=False, # whether to include the index column
   if_exists="append" # define behavior if table already exists
)

# Export product_categories table to csv without index
product_categories = pd.read_sql_query("SELECT * FROM product_categories", conn)

# Save the CSV file into the 'Tableau' folder
product_categories.to_csv("Tableau/product_categories.csv", index=False)

conn.close()

Table 5 - Products

In [16]:
# start sqlite3 connection and creation of database
conn = sqlite3.connect("superstore2.db")

cur = conn.cursor()

# Create product table
create_product_table = """
   CREATE TABLE IF NOT EXISTS product (
      product_id TEXT NOT NULL PRIMARY KEY,
      product_name TEXT NOT NULL,
      subcategory TEXT NOT NULL,
      price FLOAT NOT NULL,
      FOREIGN KEY (subcategory) REFERENCES product_categories(subcategory)
   )
"""

cur.execute(create_product_table)

cur.close()
conn.close()

In [17]:
# fill product table with data from df
product = df[["product_id", "product_name", "subcategory", "price"]].copy()
product.drop_duplicates(inplace=True)

# Check result
product

,product_id,product_name,subcategory,price
0,FUR-BO-10001798,Bush Somerset Collection Bookcase,Bookcases,130.98
1,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Chairs,243.98
2,OFF-LA-10000240,Self-Adhesive Address Labels for Typewriters b...,Labels,7.31
3,FUR-TA-10000577,Bretford CR4500 Series Slim Rectangular Table,Tables,348.21
4,OFF-ST-10000760,Eldon Fold 'N Roll Cart System,Storage,13.98
...,...,...,...,...
9521,TEC-PH-10002817,RCA ViSYS 25425RE1 Corded phone,Phones,134.99
9562,TEC-MA-10003589,Cisco 8961 IP Phone Charcoal,Machines,249.93
9604,OFF-AP-10003099,"Eureka Hand Vacuum, Bagless",Appliances,49.43
9673,TEC-PH-10002645,LG G2,Phones,499.99


In [18]:
# insert data into product_categories table
conn = sqlite3.connect("superstore2.db")

product.to_sql(
   "product", # the name of the table
   con=conn, # the connection variable
   index=False, # whether to include the index column
   if_exists="append" # define behavior if table already exists
)

# Export product table to csv without index
product = pd.read_sql_query("SELECT * FROM product", conn)

# Save the CSV file into the 'Tableau' folder
product.to_csv("Tableau/product.csv", index=False)


conn.close()

Table 6 - Calendar

In [19]:
# start sqlite3 connection and creation of database
conn = sqlite3.connect("superstore2.db")

cur = conn.cursor()

# Create product table
create_calendar_table = """
   CREATE TABLE IF NOT EXISTS product (
      order_date DATE NOT NULL PRIMARY KEY,
      day_of_week TEXT NOT NULL,
      month INT NOT NULL,
      year INT NOT NULL
      )
"""

cur.execute(create_calendar_table)

cur.close()
conn.close()

In [20]:
# fill calendar table with data from df
calendar = df[["order_date", "day_of_week", "month", "year"]].copy()
calendar.drop_duplicates(inplace=True)

# Check result
calendar

,order_date,day_of_week,month,year
0,2016-11-08,Tuesday,11,2016
2,2016-06-12,Sunday,6,2016
3,2015-10-11,Sunday,10,2015
5,2014-06-09,Monday,6,2014
12,2017-04-15,Saturday,4,2017
...,...,...,...,...
9875,2015-12-29,Tuesday,12,2015
9883,2014-04-03,Thursday,4,2014
9937,2016-06-03,Friday,6,2016
9944,2015-04-12,Sunday,4,2015


In [21]:
# insert data into product_categories table
conn = sqlite3.connect("superstore2.db")

calendar.to_sql(
   "calendar", # the name of the table
   con=conn, # the connection variable
   index=False, # whether to include the index column
   if_exists="append" # define behavior if table already exists
)

# Export calendar table to csv without index
calendar = pd.read_sql_query("SELECT * FROM calendar", conn)

# Save the CSV file into the 'Tableau' folder
calendar.to_csv("Tableau/calendar.csv", index=False)


conn.close()

Table 7a - Order header

In [22]:
# start sqlite3 connection and creation of database
conn = sqlite3.connect("superstore2.db")

cur = conn.cursor()

# Create order_header table
create_order_header_table = """
   CREATE TABLE IF NOT EXISTS order_header (
      order_id TEXT NOT NULL PRIMARY KEY,
      order_date DATE NOT NULL,
      ship_date DATE NOT NULL,
      ship_mode TEXT NOT NULL,
      customer_id TEXT NOT NULL,
      customer_address_key TEXT NOT NULL,
      FOREIGN KEY (customer_id) REFERENCES customer(customer_id),
      FOREIGN KEY (customer_address_key) REFERENCES customer_address(customer_address_key),
      FOREIGN KEY (order_date) REFERENCES order_date(calendar)
      )
"""

cur.execute(create_order_header_table)

cur.close()
conn.close()

In [23]:
# fill order table with data from df
order_header = df[["order_id", "order_date", "ship_date", "ship_mode", "customer_id", 
            "customer_address_key"]].copy()
order_header.drop_duplicates(inplace=True)

# Check result
order_header

,order_id,order_date,ship_date,ship_mode,customer_id,customer_address_key
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,CG-12520-42420-Henderson-Kentucky
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,DV-13045-90036-Los Angeles-California
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,SO-20335-33311-Fort Lauderdale-Florida
5,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,BH-11710-90032-Los Angeles-California
12,CA-2017-114412,2017-04-15,2017-04-20,Standard Class,AA-10480,AA-10480-28027-Concord-North Carolina
...,...,...,...,...,...,...
9986,CA-2016-125794,2016-09-29,2016-10-03,Standard Class,ML-17410,ML-17410-90008-Los Angeles-California
9987,CA-2017-163629,2017-11-17,2017-11-21,Standard Class,RA-19885,RA-19885-30605-Athens-Georgia
9989,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,TB-21400-33180-Miami-Florida
9990,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,DB-13060-92627-Costa Mesa-California


In [24]:
# insert data into order table
conn = sqlite3.connect("superstore2.db")

order_header.to_sql(
   "order_header", # the name of the table
   con=conn, # the connection variable
   index=False, # whether to include the index column
   if_exists="append" # define behavior if table already exists
)


# Export order_header table to csv without index
order_header = pd.read_sql_query("SELECT * FROM order_header", conn)

# Save the CSV file into the 'Tableau' folder
order_header.to_csv("Tableau/order_header.csv", index=False)


conn.close()

Table 7b - Order details

In [25]:
# start sqlite3 connection and creation of database
conn = sqlite3.connect("superstore2.db")

cur = conn.cursor()

# Create order_details table
create_order_detail_table = """
   CREATE TABLE IF NOT EXISTS order_detail (
      row_id INT NOT NULL PRIMARY KEY,
      order_id TEXT NOT NULL,
      product_id TEXT NOT NULL,
      quantity INT NOT NULL,
      sales FLOAT NOT NULL,
      discount FLOAT NOT NULL,
      profit FLOAT NOT NULL,
      FOREIGN KEY (product_id) REFERENCES product(product_id),
      FOREIGN KEY (order_id) REFERENCES order_header(order_id)
      )
"""

cur.execute(create_order_detail_table)

cur.close()
conn.close()

In [26]:
# fill order table with data from df
order_detail = df[["row_id", "order_id", "product_id", "quantity", "sales", "discount", 
            "profit"]].copy()
order_detail.drop_duplicates(inplace=True)

# Check result
order_detail

,row_id,order_id,product_id,quantity,sales,discount,profit
0,1,CA-2016-152156,FUR-BO-10001798,2,261.9600,0.00,41.9136
1,2,CA-2016-152156,FUR-CH-10000454,3,731.9400,0.00,219.5820
2,3,CA-2016-138688,OFF-LA-10000240,2,14.6200,0.00,6.8714
3,4,US-2015-108966,FUR-TA-10000577,5,957.5775,0.45,-383.0310
4,5,US-2015-108966,OFF-ST-10000760,2,22.3680,0.20,2.5164
...,...,...,...,...,...,...,...
9989,9990,CA-2014-110422,FUR-FU-10001889,3,25.2480,0.20,4.1028
9990,9991,CA-2017-121258,FUR-FU-10000747,2,91.9600,0.00,15.6332
9991,9992,CA-2017-121258,TEC-PH-10003645,2,258.5760,0.20,19.3932
9992,9993,CA-2017-121258,OFF-PA-10004041,4,29.6000,0.00,13.3200


In [27]:
# insert data into order table
conn = sqlite3.connect("superstore2.db")

order_detail.to_sql(
   "order_detail", # the name of the table
   con=conn, # the connection variable
   index=False, # whether to include the index column
   if_exists="append" # define behavior if table already exists
)


# Export order_detail table to csv without index
order_detail = pd.read_sql_query("SELECT * FROM order_detail", conn)

# Save the CSV file into the 'Tableau' folder
order_detail.to_csv("Tableau/order_detail.csv", index=False)


conn.close()

Execution of Queries to test DB

In [28]:
# Execute queries
conn = sqlite3.connect("superstore2.db")
cur = conn.cursor()

category_max_revenue = """
   SELECT DISTINCT category, SUM(order_detail.sales) AS Total_revenue
    FROM product_categories
    LEFT JOIN product ON product.subcategory = product_categories.subcategory
    LEFT JOIN order_detail ON order_detail.product_id = product.product_id
    GROUP BY category
    ORDER BY Total_revenue DESC
    LIMIT 1
"""

cur.execute(category_max_revenue)
rows = cur.fetchall()

category_max_revenue = pd.DataFrame(rows, columns=["category", "Total_revenue"])

cur.close()
conn.close()

category_max_revenue


,category,Total_revenue
0,Technology,836154.033
